In [130]:
import numpy as np
import pandas as pd
import json
import ast
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk
nltk.download('reuters')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import reuters
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
import matplotlib.pyplot as plt
import seaborn as sns
import emoji
import sklearn
import re


[nltk_data] Downloading package reuters to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [131]:
crypto = "ShibaInu"

In [132]:
dataframe = pd.read_excel(f"{crypto}_tweet_crypto.xlsx")
dataframe.head(3)

,Unnamed: 0.1,Unnamed: 0,id,text,favourite_count,retweet_count,created_at,User,place,time,high,low,open,volume_from,volume_to,close
0,0,1733,1.509960e+18,What happened today - One of our clients sent ...,14,7,2022-04-01 18:26:59,"{'id': 103292437, 'id_str': '103292437', 'name...",NaN,2022-04-01 18:27:00,0.000026,0.000026,0.000026,3.176552e+09,83179.86,0.000026
1,1,1732,1.509960e+18,@illuviumio I am super bullish on Bullypit. Th...,0,0,2022-04-01 18:27:15,"{'id': 1163971113375993856, 'id_str': '1163971...",NaN,2022-04-01 18:28:00,0.000026,0.000026,0.000026,4.439739e+09,116320.86,0.000026
2,2,1731,1.509960e+18,Found out yesterday that $cult dao is from the...,1,0,2022-04-01 18:27:37,"{'id': 1507873925690269696, 'id_str': '1507873...",NaN,2022-04-01 18:28:00,0.000026,0.000026,0.000026,4.439739e+09,116320.86,0.000026


In [133]:
dataframe.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'text', 'favourite_count',
       'retweet_count', 'created_at', 'User', 'place', 'time', 'high', 'low',
       'open', 'volume_from', 'volume_to', 'close'],
      dtype='object')

In [134]:
arr = np.squeeze(np.array(dataframe["User"].apply(pd.Series)))
dataArray = []
for i in tqdm(range(len(arr))):
    dict = ast.literal_eval(arr[i])
    json_string = json.dumps(dict) 
    newData = json.loads(json_string)
    f1 = pd.json_normalize(newData)
    dataArray.append(f1)

100%|██████████| 42857/42857 [01:23<00:00, 511.20it/s]


In [135]:
user_id_str_list = []
user_name_list = []
user_f_count_list = []
user_verif = []
for i in tqdm(range(len(arr))):
    id_str = dataArray[i]['id_str']
    name = dataArray[i]['name']
    f_cnt = dataArray[i]['followers_count']
    verif = dataArray[i]['verified']
    user_id_str_list.append(id_str)
    user_name_list.append(name)
    user_f_count_list.append(f_cnt)
    user_verif.append(verif)
    

100%|██████████| 42857/42857 [00:09<00:00, 4469.27it/s]


In [136]:
dataframe['user_id_str'] = np.array(user_id_str_list)
dataframe['user_name'] = np.array(user_name_list)
dataframe['user_followers_count'] = np.array(user_f_count_list)
dataframe['user_verified'] = np.array(user_verif)

In [137]:
dataframe.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'text', 'favourite_count',
       'retweet_count', 'created_at', 'User', 'place', 'time', 'high', 'low',
       'open', 'volume_from', 'volume_to', 'close', 'user_id_str', 'user_name',
       'user_followers_count', 'user_verified'],
      dtype='object')

In [138]:
# newUserData = pd.concat(dataArray)
# newUserData = pd.DataFrame(newUserData)
# newUserData.head(3)

In [139]:
# newUserData.shape

In [140]:
# newUserData.columns

In [141]:
# for col in newUserData.columns:
#     newName = "user_"+col
#     newUserData = newUserData.rename(columns = {col: newName})


In [142]:
# newUserData.columns

In [143]:
# for col in newUserData.columns:
#     dataframe[col] = np.array(newUserData[col])

In [144]:
dataframe.shape

(42857, 20)

In [145]:
reuters = set(nltk.corpus.reuters.words())
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

In [146]:
def find_retweeted(tweet):
    '''This function will extract the twitter handles of retweed people'''
    return re.findall('(?<=RT\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)
    
def find_mentioned(tweet):
    '''This function will extract the twitter handles of people mentioned in the tweet'''
    return re.findall('(?<!RT\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)  

def find_hashtags(tweet):
    '''This function will extract hashtags'''
    return re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', tweet) 


def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = []
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    
    tweet = ' '.join(tweet_token_list)
    
    return tweet
    
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
    
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, EMOTICONS_EMO[emot].replace(" ","_"))
    return text

def convert_emojis(tweet):
    tweet = emoji.demojize(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    return tweet

In [147]:
dataframe['text'] = dataframe['text'].apply(str)

In [148]:
dataframe['hashtags'] = dataframe.text.apply(find_hashtags)
dataframe['mentioned'] = dataframe.text.apply(find_mentioned)
dataframe['text'] = dataframe.text.apply(convert_emoticons)

In [149]:
dataframe['clean_tweet'] = dataframe.text.apply(clean_tweet)
dataframe['changed_text'] = dataframe.clean_tweet.apply(convert_emojis)

In [150]:
dataframe.shape

(42857, 24)

In [151]:
dataframe.to_csv(f'preprocessed_data/{crypto}_combined_preproc.csv', index=False)

In [152]:
kf = pd.read_csv(f'preprocessed_data/{crypto}_combined_preproc.csv')
print(kf.shape)


(42857, 24)
